# Snyk Analysis Error Troubleshooting Guide

This notebook explains the error encountered during Snyk dependency analysis and the solution implemented.

## Error Context
When running `python -m src.interface.cli --xlsx`, the analysis encounters missing packages in the temporary pipgrip resolution environment.


## The Error Message Breakdown

The error occurs in three parts:

### 1. SyntaxWarning: Invalid Escape Sequence
```
SyntaxWarning: invalid escape sequence '\s'
  for sub_exp in re.split("\\s*(?:and|or)\\s*", cond_text):
```
This is a minor warning from pipgrip's internal pip resolver. It's harmless but indicates outdated code.

### 2. UserWarning: Deprecated pkg_resources
```
UserWarning: pkg_resources is deprecated as an API
The pkg_resources package is slated for removal as early as 2025-11-30.
```
This warns that `pkg_resources` (from setuptools) is deprecated. Pin setuptools to <81 if needed.

### 3. Critical Error: Missing Packages
```
Required packages missing: ipykernel, comm, debugpy, ipython, ...
Please run `pip install -r C:\...\requirements.txt`
If the issue persists try again with --skip-unresolved.
```
This is the main problem: pipgrip's temporary environment lacks dependencies needed for resolution.


## Root Cause Analysis

When pipgrip resolves dependencies, it:
1. Creates a temporary isolated environment
2. Tries to install and inspect packages in that environment
3. Recursively resolves their dependencies
4. Returns the complete dependency tree as JSON

The error occurs because **some packages cannot be fully resolved in the isolated environment** due to:
- Missing system dependencies
- Platform-specific requirements
- C extensions that need compilation
- Environment markers that don't match

### Why This Happens
- `ipykernel`, `jupyter-client`, etc. are dev/optional dependencies of some packages
- They may not be installable in the temporary resolution environment
- Pipgrip **fails hard** by default when it encounters unresolvable packages


## The Solution: `--skip-unresolved` Flag

### What Changed
**File:** `src/infrastructure/adapters/dependency_resolver_adapter.py`

**Before:**
```python
cmd = ["pipgrip", "--tree-json-exact", package]
```

**After:**
```python
cmd = ["pipgrip", "--tree-json-exact", "--skip-unresolved", package]
```

### What `--skip-unresolved` Does
- **Tells pipgrip to continue** even when some packages cannot be resolved
- **Returns partial results** instead of failing completely
- **Includes resolved dependencies** and skips the ones that couldn't be installed
- **Allows analysis to proceed** with "best effort" dependency information

### Why This Works
1. Most packages CAN be resolved (numpy, requests, etc.)
2. Only optional/dev packages in isolated env fail (jupyter, ipython, etc.)
3. We only need the dependency tree, not a working installation
4. Partial dependency information is better than complete failure


## Implementation Details

### Code Location
- **File:** `e:\Repos\pypi\src\infrastructure\adapters\dependency_resolver_adapter.py`
- **Method:** `_resolve_single_package_with_pipgrip()`
- **Line:** 282

### The Change
```python
async def _resolve_single_package_with_pipgrip(self, package: str, timeout_sec: int = 300) -> Dict[str, Any]:
    """Resolve a single package using pipgrip."""
    try:
        env = os.environ.copy()
        env["PIP_PREFER_BINARY"] = "1"
        env["PIP_DISABLE_PIP_VERSION_CHECK"] = "1"

        # NEW FLAG ADDED: --skip-unresolved
        cmd = ["pipgrip", "--tree-json-exact", "--skip-unresolved", package]
        
        # Rest of the method remains unchanged...
        proc = await asyncio.create_subprocess_exec(
            *cmd,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
            cwd=str(Path.cwd()),
            env=env
        )
        # ...
```

### Pipgrip Command Flags Explained
- `--tree-json-exact`: Output complete dependency tree as JSON with exact versions
- `--skip-unresolved`: Skip packages that cannot be resolved (NEW)
- `package`: The package to resolve (e.g., "requests>=2.20")


## Testing the Fix

### Expected Behavior After Fix
1. ✅ Analysis **does not fail** on missing packages
2. ✅ Pipgrip returns **partial dependency tree** (resolved + skipped)
3. ✅ Snyk analysis **completes** and generates xlsx report
4. ✅ xlsx contains **all resolvable packages and their dependencies**

### How to Verify
```bash
# Run the analysis with the fixed code
cd e:\Repos\pypi
python -m src.interface.cli --xlsx

# Check output:
# - Should generate consolidated_report.xlsx
# - Should include Dependencias Directas and Dependencias Transitivas
# - NO timeout or "Required packages missing" error
```

### What to Expect in xlsx
- **Packages column:** All packages found (both production and dev)
- **Dependencias Directas:** Production dependencies (from PEP 508)
- **Dependencias Transitivas:** Development/optional dependencies
- **Some packages may show empty dependencies** if they were skipped by pipgrip


## Alternative Solutions (Not Implemented)

### Option 1: Install Missing Packages (Not Recommended)
```bash
pip install ipykernel comm debugpy ipython decorator ipython-pygments-lexers ...
```
**Problem:** 
- This would pollute the main environment
- Jupyter packages not needed for dependency analysis
- Could conflict with existing packages

### Option 2: Use Different Resolver (pip instead of pipgrip)
```python
cmd = ["pip", "download", "--dry-run", package]
```
**Problem:**
- pip doesn't support `--tree-json-exact` output format
- Would need custom JSON parsing
- Slower than pipgrip

### Option 3: Pre-resolve in Clean Virtual Environment (Complex)
```bash
python -m venv /tmp/resolve_env
source /tmp/resolve_env/bin/activate
pip install package
pip show --json package
```
**Problem:**
- Much slower (create venv for each package)
- Resource intensive
- Added complexity

### Why `--skip-unresolved` is Best
✅ Minimal change (single flag)
✅ Fast (no venv creation)
✅ Reliable (pipgrip design intended for this)
✅ Acceptable trade-off (partial info > no info)


## Summary of Changes

| Aspect | Details |
|--------|---------|
| **File Modified** | `src/infrastructure/adapters/dependency_resolver_adapter.py` |
| **Method** | `_resolve_single_package_with_pipgrip()` |
| **Change** | Added `--skip-unresolved` flag to pipgrip command |
| **Impact** | Analysis continues despite unresolvable packages |
| **Test Command** | `python -m src.interface.cli --xlsx` |
| **Expected Outcome** | xlsx report generated with dependency information |

## Related Improvements in This Session

This fix is part of a larger refactoring that includes:
1. ✅ Fixed empty Dependencias columns (dependency graph extraction)
2. ✅ Separated production vs development dependencies (PEP 508 parsing)
3. ✅ Fixed Snyk analysis unresolved packages (--skip-unresolved flag)

All three improvements work together to provide accurate, complete dependency analysis.
